#### **Bước 1. Phân tích các phương thức có trong chương trinh**


In [6]:
from random import randint
import sys

##### ***1.1. generator(g, x, p)***
Tính toán: $(g^x) \mod p $

In [7]:
def generator(g, x, p):
    return pow(g, x) % p

##### ***1.2. encrypt(plaintext, key)***
Phương thức mã hóa một chuỗi văn bản (plaintext) bằng cách nhân mã ASCII của từng ký tự với một khóa (key) và một hằng số (311).
1. Lặp từng ký tự trong `plaintext`
2. Mỗi ký tự --> ASCII: $c = \text{ord}(c) \times key \times 311$
3. Mỗi phần tử lưu vào mảng `cipher`, trả về mảng `cipher`.

In [8]:
def encrypt(plaintext, key):
    cipher = []
    for char in plaintext:
        cipher.append(((ord(char) * key * 311)))
    return cipher

##### ***1.3. is_prime(p)***
Phương thức kiểm tra `p` có phải là số nguyên tố hay không?

In [9]:
def is_prime(p):
    v = 0
    for i in range(2, p + 1):
        if p % i == 0:
            v = v + 1
    if v > 1:
        return False
    else:
        return True

##### ***1.4. dynamic_xor_encrypt(plaintext, text_key)***
Phương thức mã hóa văn bản sử dụng phương pháp **`XOR`** với một khóa chuỗi `ext_key`.
1. Duyệt từng ký tự theo thứ tự ngược lại trong `plaintext`, lấy chỉ số: `i`
2. Lấy ký tự tương ứng từ `text_key` theo chỉ số `i`.
3. Thực hiện phép **`XOR`** giữa mã ASCII của `char` và `key_char`, sau đó chuyển đổi giá trị trở lại thành ký tự --> Lưu vào `cipher_text`

In [10]:
def dynamic_xor_encrypt(plaintext, text_key):
    cipher_text = ""
    key_length = len(text_key)
    for i, char in enumerate(plaintext[::-1]):
        key_char = text_key[i % key_length]
        encrypted_char = chr(ord(char) ^ ord(key_char))
        cipher_text += encrypted_char
    return cipher_text

##### ***1.5. test(plain_text, text_key)***
Phương thức thực hiện quy trình mã hóa văn bản sử dụng các phương thức khác nhau.
1. Cho `p` và `g` là hai số nguyên tố
2. Lấy ngẫu nhiên $a \in (p-10, p)$ và $b \in (g-10, g)$
3. Sinh ra hai bộ tạo `u`, `v` với: $u = \text{generator}(g, a, p)$ và $v = \text{generator}(g, b, p)$
4. Sinh ra hai bộ tạo `key` và `b_key` với: $key = \text{generator}(v, a, p)$ và $b\_key = \text{generator}(u, b, p)$. Điều này nghĩa là:
$ key = (g^b)^a \mod p = b\_key$
5. Tạo một $share\_key = key$
6. Thực hiện: `semi_cipher = dynamic_xor_encrypt(plain_text, text_key)`
7. Thực hiện: `cipher = encrypt(semi_cipher, shared_key)`
8. Trả về `cipher`

In [11]:
def test(plain_text, text_key):
    p = 97
    g = 31
    if not is_prime(p) and not is_prime(g):
        print("Enter prime numbers")
        return
    a = randint(p-10, p)
    b = randint(g-10, g)
    print(f"a = {a}")
    print(f"b = {b}")
    u = generator(g, a, p)
    v = generator(g, b, p)
    key = generator(v, a, p)
    b_key = generator(u, b, p)
    shared_key = None
    if key == b_key:
        shared_key = key
    else:
        print("Invalid key")
        return
    semi_cipher = dynamic_xor_encrypt(plain_text, text_key)
    cipher = encrypt(semi_cipher, shared_key)
    print(f'cipher is: {cipher}')

#### **Bước 2. Hiểu chương trình**
Chương trình cho trước 02 biến: $a = 94$ và $b = 21$. Các tham số $p=97$ và $g=31$

**Yêu cầu:** Phải giải mã được thông điệp message từ biến `cipher` cho trước

1. Ta có thể tính được `share_key`: 
$$share\_key = (g^a)^b \mod p$$
2. Thực hiện chia ngược lại bằng hàm decrypt, ta được một biến `semi_decrypt`: 
$$semi\_decrypt = \text{round}\left(\frac{item}{311 \times share\_key}\right)$$
3. Chuyển cipher thành các ký tự rồi tiếp tục sử dụng hàm `dynamic_xor_encrypt`, vì bản chất $$a \oplus b \oplus b = a$$


In [30]:
# Đọc nội dung từ tệp
with open('enc_flag', 'r') as file:
    lines = file.readlines()

# Khởi tạo biến
a = None
b = None
cipher = []

# Trích xuất giá trị từ các dòng
for line in lines:
    if line.startswith('a ='):
        a = int(line.split('=')[1].strip())
    elif line.startswith('b ='):
        b = int(line.split('=')[1].strip())
    elif line.startswith('cipher is:'):
        cipher = eval(line.split(':')[1].strip())

p = 97
g = 31

key = generator(generator(g, b, p), a, p)

# Định nghĩa một phương thức decrypt():
def decrypt(cipher:list, key: int) -> str:
    s = []
    plaintext = ""
    for item in cipher:
        num = item // (key * 311)
        s.append(num)
        char = chr(num)
        plaintext += char
    print(s)
    return plaintext, s

semi_plaintext, plaintext_arr = decrypt(cipher, key)
def dynamic_xor_decrypt(plaintext_arr, text_key):
    cipher_text = ""
    key_length = len(text_key)
    for i, char in enumerate(plaintext_arr):
        key_char = text_key[i % key_length]
        encrypted_char = chr(char ^ ord(key_char))
        cipher_text += encrypted_char
    cipher_text = cipher_text[::-1]
    return cipher_text

plaintext = dynamic_xor_decrypt(plaintext_arr, "trudeau")
print(plaintext)

print(key)

[9, 68, 66, 93, 3, 80, 65, 22, 74, 42, 0, 83, 21, 5, 68, 0, 22, 86, 1, 62, 24, 27, 6, 6, 17, 6, 26, 51, 32, 49, 26, 7, 12, 17]
picoCTF{custom_d2cr0pt6d_8b41f976}
47
